# 15d Prediction

#### With model pretrained in Fund_Analysis_Model file, this streamlined file is meant for active trading use and has no other functionality. See Fund_Analysis_Model for all analysis, model generation, training, etc.

In [3]:
import pandas as pd 
import numpy as np
import datetime
from datetime import date
import time
import pickle
import math
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, mean_absolute_error, mean_absolute_percentage_error
from xgboost import XGBRegressor
import xgboost as xgb
import sklearn as skl
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Bidirectional
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.metrics import RootMeanSquaredError, MeanSquaredError
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import keras.backend as K
import os
import tensorflow as tf

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)

with open('reg_15.pickle', 'rb') as file:
    reg15 = pickle.load(file)

In [2]:
dow30 = pd.read_csv('All_Data.csv')
dow30.rename(columns = {'date':'Date'}, inplace = True)
dow30 = dow30.drop(columns=['index'])
dow30['Date'] = pd.to_datetime(dow30['Date'])
dow30 = dow30.drop(columns=['high', 'low', 'open'])
dow30 = dow30.sort_values(by=['Stock', 'Date'], ascending=True)
dow30 = dow30.reset_index().drop(columns=['index'])

dates = dow30['Date']
stocks = dow30['Stock']
indices_to_dates = dates.to_dict()
indices_to_stocks = stocks.to_dict()
act_returns_mapping15 = dow30['15d_Return'].to_dict()
act_returns_mapping30 = dow30['30d_Return'].to_dict()
act_returns_mapping60 = dow30['60d_Return'].to_dict()
price_mapping = dow30['close'].to_dict()

dow30

,Date,10YBond,30dBondRet,VIX_high,spy_close,spy_5dret,spy_10dret,spy_15dret,surprise_%,expected_growth,previous_surprise,days_after_earn_report,dividends_change,prev_div_change,days_after_divid_report,close,volume,divCash,debtCurrent,taxAssets,investmentsCurrent,totalAssets,acctPay,accoci,inventory,totalLiabilities,acctRec,intangibles,ppeq,deferredRev,cashAndEq,assetsNonCurrent,taxLiabilities,investments,equity,retainedEarnings,deposits,assetsCurrent,investmentsNonCurrent,debt,debtNonCurrent,liabilitiesNonCurrent,liabilitiesCurrent,sharesBasic,longTermDebtEquity,shareFactor,bookVal,roa,currentRatio,roe,grossMargin,piotroskiFScore,epsQoQ,revenueQoQ,profitMargin,rps,bvps,ncfi,capex,ncfx,ncff,sbcomp,ncf,payDiv,businessAcqDisposals,issrepayDebt,issrepayEquity,investmentsAcqDisposals,freeCashFlow,ncfo,depamor,ebitda,netIncComStock,epsDil,consolidatedIncome,nonControllingInterests,shareswaDil,intexp,rnd,eps,netIncDiscOps,grossProfit,shareswa,opex,ebt,netinc,revenue,ebit,taxExp,opinc,sga,costRev,prefDVDs,quarter,year,marketCap,enterpriseVal,peRatio,pbRatio,trailingPEG1Y,Debt-to-Equity_Ratio,DividendsYield,PayoutRatio,Acc_Rec_Pay_Ration,Earnings_per_stock,15d_Return,30d_Return,60d_Return,Stock
0,2019-03-11,2.643,-0.039956,16.43,260.370637,NaN,NaN,NaN,0.0096,-0.438095,0.0429,41.0,0.158730,0.105263,314.0,43.229318,128044136.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.068977,0.159754,0.024331,AAPL
1,2019-03-12,2.605,-0.050656,14.70,261.352498,NaN,NaN,NaN,0.0096,-0.438095,0.0429,42.0,0.158730,0.105263,315.0,43.715014,129870336.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.072467,0.145100,0.027822,AAPL
2,2019-03-13,2.611,-0.037242,14.05,263.082442,NaN,NaN,NaN,0.0096,-0.438095,0.0429,43.0,0.158730,0.105263,316.0,43.908325,124130096.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.075065,0.129712,0.050534,AAPL
3,2019-03-14,2.630,-0.024119,13.84,262.914123,NaN,NaN,NaN,0.0096,-0.438095,0.0429,44.0,0.158730,0.105263,317.0,44.396437,94318032.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.065096,0.111958,0.052262,AAPL
4,2019-03-15,2.593,-0.015939,13.28,264.207467,NaN,NaN,NaN,0.0096,-0.438095,0.0429,45.0,0.158730,0.105263,318.0,44.973956,156171648.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.058457,0.099345,0.050778,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

## Feature Generation

### Percentage from monthly high, RSI

In [5]:
def get_monthly_highs(df):
    df = df.sort_values(by=['Stock', 'Date'], ascending=True)
    row = 0
    curr_month = df['Date'].loc[0].month
    highs=[]
    while (row < len(df)):
        closes = []
        count=0
        while (row<len(df)) and (df['Date'].loc[row].month == curr_month):
            closes.append(df['close'].loc[row])
            row+=1
            count+=1
        for i in range(0,count):
            highs.append(max(closes))
        if (row < len(df)):
            curr_month = df['Date'].loc[row].month
    
    return highs    
        

In [6]:
def get_rsi(prices):
    periods = 14
    ema = True
    close_delta = prices.diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
        # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [7]:
highs = np.array(get_monthly_highs(dow30))
dow30['Percent_from_mth_high'] = (highs / dow30['close']) - 1

RSI = dow30.groupby('Stock').apply(lambda x: get_rsi(x['close']))
rsi = np.array(RSI)
dow30['RSI'] = rsi

dow30orig = dow30.copy()

dow30

,Date,10YBond,30dBondRet,VIX_high,spy_close,spy_5dret,spy_10dret,spy_15dret,surprise_%,expected_growth,previous_surprise,days_after_earn_report,dividends_change,prev_div_change,days_after_divid_report,close,volume,divCash,debtCurrent,taxAssets,investmentsCurrent,totalAssets,acctPay,accoci,inventory,totalLiabilities,acctRec,intangibles,ppeq,deferredRev,cashAndEq,assetsNonCurrent,taxLiabilities,investments,equity,retainedEarnings,deposits,assetsCurrent,investmentsNonCurrent,debt,debtNonCurrent,liabilitiesNonCurrent,liabilitiesCurrent,sharesBasic,longTermDebtEquity,shareFactor,bookVal,roa,currentRatio,roe,grossMargin,piotroskiFScore,epsQoQ,revenueQoQ,profitMargin,rps,bvps,ncfi,capex,ncfx,ncff,sbcomp,ncf,payDiv,businessAcqDisposals,issrepayDebt,issrepayEquity,investmentsAcqDisposals,freeCashFlow,ncfo,depamor,ebitda,netIncComStock,epsDil,consolidatedIncome,nonControllingInterests,shareswaDil,intexp,rnd,eps,netIncDiscOps,grossProfit,shareswa,opex,ebt,netinc,revenue,ebit,taxExp,opinc,sga,costRev,prefDVDs,quarter,year,marketCap,enterpriseVal,peRatio,pbRatio,trailingPEG1Y,Debt-to-Equity_Ratio,DividendsYield,PayoutRatio,Acc_Rec_Pay_Ration,Earnings_per_stock,15d_Return,30d_Return,60d_Return,Stock,Percent_from_mth_high,RSI
0,2019-03-11,2.643,-0.039956,16.43,260.370637,NaN,NaN,NaN,0.0096,-0.438095,0.0429,41.0,0.158730,0.105263,314.0,43.229318,128044136.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.068977,0.159754,0.024331,AAPL,0.090497,NaN
1,2019-03-12,2.605,-0.050656,14.70,261.352498,NaN,NaN,NaN,0.0096,-0.438095,0.0429,42.0,0.158730,0.105263,315.0,43.715014,129870336.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.072467,0.145100,0.027822,AAPL,0.078382,NaN
2,2019-03-13,2.611,-0.037242,14.05,263.082442,NaN,NaN,NaN,0.0096,-0.438095,0.0429,43.0,0.158730,0.105263,316.0,43.908325,124130096.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.075065,0.129712,0.050534,AAPL,0.073634,NaN
3,2019-03-14,2.630,-0.024119,13.84,262.914123,NaN,NaN,NaN,0.0096,-0.438095,0.0429,44.0,0.158730,0.105263,317.0,44.396437,94318032.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.065096,0.111958,0.052262,AAPL,0.061830,NaN
4,2019-03-15,2.593,-0.015939,13.28,264.207467,NaN,NaN,NaN,0.0096,-0.438095,0.0429,45.0,0.158730,0.105263,318.0,44.973956,156171648.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.058457,0.099345,0.050778,AAPL,0.048195,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

### Forward fill last day or two of fundamental data if needed

In [9]:
# make predictions df, with any NaN values forward filled from previous day in the vicinity
dow30_lastdays = dow30orig.loc[dow30orig['Date'] > pd.to_datetime('2023-02-26')] 
dow30_lastdays.ffill(axis=0, inplace=True)
dow30_lastdays

C:\Users\HP\AppData\Local\Temp\ipykernel_6364\1480192896.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dow30_lastdays.ffill(axis=0, inplace=True)


,Date,10YBond,30dBondRet,VIX_high,spy_close,spy_5dret,spy_10dret,spy_15dret,surprise_%,expected_growth,previous_surprise,days_after_earn_report,dividends_change,prev_div_change,days_after_divid_report,close,volume,divCash,debtCurrent,taxAssets,investmentsCurrent,totalAssets,acctPay,accoci,inventory,totalLiabilities,acctRec,intangibles,ppeq,deferredRev,cashAndEq,assetsNonCurrent,taxLiabilities,investments,equity,retainedEarnings,deposits,assetsCurrent,investmentsNonCurrent,debt,debtNonCurrent,liabilitiesNonCurrent,liabilitiesCurrent,sharesBasic,longTermDebtEquity,shareFactor,bookVal,roa,currentRatio,roe,grossMargin,piotroskiFScore,epsQoQ,revenueQoQ,profitMargin,rps,bvps,ncfi,capex,ncfx,ncff,sbcomp,ncf,payDiv,businessAcqDisposals,issrepayDebt,issrepayEquity,investmentsAcqDisposals,freeCashFlow,ncfo,depamor,ebitda,netIncComStock,epsDil,consolidatedIncome,nonControllingInterests,shareswaDil,intexp,rnd,eps,netIncDiscOps,grossProfit,shareswa,opex,ebt,netinc,revenue,ebit,taxExp,opinc,sga,costRev,prefDVDs,quarter,year,marketCap,enterpriseVal,peRatio,pbRatio,trailingPEG1Y,Debt-to-Equity_Ratio,DividendsYield,PayoutRatio,Acc_Rec_Pay_Ration,Earnings_per_stock,15d_Return,30d_Return,60d_Return,Stock,Percent_from_mth_high,RSI
999,2023-02-27,3.922,0.137141,22.02,397.73,-0.023400,-0.025267,-0.035455,0.0238,0.116279,0.0526,123.0,-0.260870,0.045455,69.0,147.92,44998470.0,0.00,1.148300e+10,0.000000e+00,3.082000e+10,3.467470e+11,5.791800e+10,-1.291200e+10,6.820000e+09,2.900200e+11,5.418000e+10,0.000000e+00,4.295100e+10,7.992000e+09,2.053500e+10,2.179700e+11,0.000000e+00,1.449150e+11,5.672700e+10,3.240000e+09,0.000000e+00,1.287770e+11,1.140950e+11,1.111100e+11,9.962700e+10,1.527340e+11,1.372860e+11,1.590812e+10,1.756254,1.000,5.672700e+10,0.274570,0.938020,1.634503,0.429623,4.0,-0.104265,-0.054790,0.429623,7.364416,3.565915,-1.445000e+09,-3.787000e+09,0.000000e+00,-3.556300e+10,2.905000e+09,-3.003000e+09,-3.768000e+09,0.000000e+00,-9.615000e+09,-1.947500e+10,2.483000e+09,3.021800e+10,3.400500e+10,2.916000e+09,3.954200e+10,2.999800e+10,1.88,2.999800e+10,0.0,1.595572e+10,1.003000e+09,7.709000e+09,1.89,0.0,5.033200e+10,1.589272e+10,1.431600e+10,3.562300e+10,2.999800e+10,1.171540e+11,3.662600e+10,5.625000e+09,3.601600e+10,6.607000e+09,6.682200e+10,0.0,1.0,2023.0,2.340382e+12,2.430957e+12,24.591338,41.256937,-2.358533,1.195597,-0.001610,-0.074863,0.935460,0.012710,NaN,NaN,NaN,AAPL,0.050095,51.171425
1000,2023-02-28,3.916,0.115352,21.37,396.26,-0.007091,-0.040138,-0.033111,0.0238,0.116279,0.0526,124.0,-0.260870,0.045455,70.0,147.41,50546998.0,0.00,1.148300e+10,0.000000e+00,3.082000e+10,3.467470e+11,5.791800e+10,-1.291200e+10,6.820000e+09,2.900200e+11,5.418000e+10,0.000000e+00,4.295100e+10,7.992000e+09,2.053500e+10,2.179700e+11,0.000000e+00,1.449150e+11,5.672700e+10,3.240000e+09,0.000000e+00,1.287770e+11,1.140950e+11,1.111100e+11,9.962700e+10,1.527340e+11,1.372860e+11,1.590812e+10,1.756254,1.000,5.672700e+10,0.274570,0.938020,1.634503,0.429623,4.0,-0.104265,-0.054790,0.429623,7.364416,3.565915,-1.445000e+09,-3.787000e+09,0.000000e+00,-3.556300e+10,2.905000e+09,-3.003000e+09,-3.768000e+09,0.000000e+00,-9.615000e+09,-1.947500e+10,2.483000e+09,3.021800e+10,3.400500e+10,2.916000e+09,3.954200e+10,2.999800e+10,1.88,2.999800e+10,0.0,1.595572e+10,1.003000e+09,7.709000e+09,1.89,0.0,5.033200e+10,1.589272e+10,1.431600e+10,3.562300e+10,2.999800e+10,1.171540e+11,3.662600e+10,5.625000e+09,3.601600e+10,6.607000e+09,6.682200e+10,0.0,1.0,2023.0,2.332313e+12,2.422888e+12,24.506552,41.114691,-2.350401,1.195597,-0.001610,-0.074863,0.935460,0.012710,NaN,NaN,NaN,AAPL,0.053728,50.127405
2000,2023-02-27,3.922,0.137141,22.02,397.73,-0.023400,-0.025267,-0.035455,0.0609,-0.165957,0.0568,116.0,0.102273,0.100000,451.0,234.45,1723134.0,0.00,1.591000e+09,0.000000e+00,1.676000e+09,6.512100e+10,1.572000e+09,-2.310000e+08,4.930000e+09,6.146000e+10,5.563000e+09,3.160900e+10,5.427000e+09,0.000000e+00,7.629000e+09,4.293500e+10,5.757000e+09,1.676000e+09,3.661000e+09,-2.862200e+10,0.000000e+00

In [15]:
preds = reg15.predict(dow30_lastdays.drop(columns=['15d_Return', '30d_Return', '60d_Return','Date', 'Stock']))

Predictions = dow30_lastdays.copy()
Predictions = Predictions[['15d_Return','Date']]
Predictions['Predicted 15d_Return'] = preds
Predictions['Date'] = Predictions.index.map(indices_to_dates)
Predictions['Stock'] = Predictions.index.map(indices_to_stocks)
Predictions['Date'] = pd.to_datetime(Predictions['Date'])
Results = Predictions.sort_values(by=['Predicted 15d_Return', 'Date'], ascending=False)
Results.head(10)

,15d_Return,Date,Predicted 15d_Return,Stock
28026,NaN,2023-02-27,0.134321,VZ
9007,NaN,2023-02-27,0.128076,DIS
15013,NaN,2023-02-27,0.115284,INTC
28027,NaN,2023-02-28,0.113328,VZ
15014,NaN,2023-02-28,0.105826,INTC
12010,NaN,2023-02-27,0.104545,HD
12011,NaN,2023-02-28,0.103801,HD
8007,NaN,2023-02-28,0.094964,CVX
9008,NaN,2023-02-28,0.094359,DIS
22021,NaN,2023-02-28,0.090800,MSFT
